In [2]:
import numpy as np
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from tensorflow.keras.experimental import SequenceFeatures

from DataLoader import CSVLoader
import pandas as pd

In [3]:
col_type = {
    "aid": "string",
    "click_adid": "stringv",
    "age": "int64",
    # "sex": ,
    "cp": "int64",
    # "trestbps": "int64",
    "chol": "int64",
    # "fbs": "int64",
    # "restecg": "int64",
    # "thalach": "int64",
    # "exang": "int64",
    "oldpeak": "float32",
    # "slope": "int64",
    # "ca": "int64",
    # "thal": "string",
    "target": "int64",
}
ds = CSVLoader("data/*", col_type, 
            batch_size=2, shuffle=False,
            num_epochs=None,
            varlen_ragged=False,
            varlen_max_len=8,
            label_name='target').load()
pd.read_csv('data/toy.csv', usecols=list(col_type))

,aid,click_adid,age,cp,chol,oldpeak,target
0,dgadfadsfag,sdfgad|adgas|adgasdfd|asdgadsg|asdgadsfa|Asdga...,63,1,233,2.3,0
1,sdfgad,sdfgad|adgas|adgasdfd,67,4,286,1.5,1
2,adgas,asdgadsg|asdgadsfa|Asdgasd|Asdga,67,4,229,2.6,0
3,adgasdfd,adgasdfd|asdgadsg|asdgadsfa,37,3,250,3.5,0
4,asdgadsg,Asdgasd|Asdga,41,2,204,1.4,0
5,asdgadsfa,sdfgad,56,2,236,0.8,0
6,Asdgasd,adgas|adgasdfd,62,4,268,3.6,1
7,Asdga,sdfgad|adgas|adgasdfd|asdgadsg|asdgadsfa,57,4,354,0.6,0


In [4]:
example_batch, label = next(iter(ds))
def demo_dense(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

def demo_seq(feature_column):
  feature_layer = SequenceFeatures(feature_column)
  # SequenceFeatures(feature_column) return 2 tensor (sequence_value, sequence_length)
  print(feature_layer(example_batch)[0].numpy())

def demo_seq_in(example_batch, feature_column):
  feature_layer = SequenceFeatures(feature_column)
  print(feature_layer(example_batch))

example_batch

OrderedDict([('aid',
              <tf.Tensor: shape=(2,), dtype=string, numpy=array([b'dgadfadsfag', b'sdfgad'], dtype=object)>),
             ('click_adid', <tf.Tensor: shape=(2, 8), dtype=string, numpy=
              array([[b'sdfgad', b'adgas', b'adgasdfd', b'asdgadsg', b'asdgadsfa',
                      b'Asdgasd', b'Asdga', b''],
                     [b'sdfgad', b'adgas', b'adgasdfd', b'', b'', b'', b'', b'']],
                    dtype=object)>),
             ('age',
              <tf.Tensor: shape=(2,), dtype=int64, numpy=array([63, 67])>),
             ('cp', <tf.Tensor: shape=(2,), dtype=int64, numpy=array([1, 4])>),
             ('chol',
              <tf.Tensor: shape=(2,), dtype=int64, numpy=array([233, 286])>),
             ('oldpeak',
              <tf.Tensor: shape=(2,), dtype=float32, numpy=array([2.3, 1.5], dtype=float32)>)])

In [5]:
embedding_size = 3
cp_backets = 5
aid_backets = 5

In [6]:
# int map to [0, 400) and embedding
cp = feature_column.embedding_column(
    feature_column.categorical_column_with_identity(
      "cp", cp_backets), embedding_size)

demo_dense(cp)

[[ 0.7971227  -0.03460998  0.67342865]
 [-0.00436206  0.21655248 -0.58231163]]


In [7]:
aid = feature_column.indicator_column(
    feature_column.categorical_column_with_hash_bucket(
      "aid", aid_backets), )

demo_dense(aid)

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[[0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]


In [8]:
aid = feature_column.embedding_column(
    feature_column.categorical_column_with_hash_bucket(
      "aid", aid_backets), embedding_size)

demo_dense(aid)

[[ 0.39268118 -0.3036767  -0.15580931]
 [ 0.32843956  0.45055446  0.24686807]]


In [9]:
age = feature_column.numeric_column('age')
demo_dense(age)

[[63.]
 [67.]]


In [10]:
age = feature_column.indicator_column(
            feature_column.bucketized_column(
                feature_column.numeric_column('age')
                , boundaries = [0, 50, 60, 100]), )

demo_dense(age)

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[[0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]]


In [11]:
age_bucket = feature_column.bucketized_column(
                feature_column.numeric_column('age')
                , boundaries = [0, 50, 60, 100])

bucket_size = 100

cross = feature_column.indicator_column(
            feature_column.crossed_column(
                ["aid", age_bucket],
                bucket_size
            ), )

demo_dense(cross)

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]]


In [12]:
backet_seq = 10
click_adid = feature_column.sequence_categorical_column_with_hash_bucket(
    'click_adid',
    backet_seq
)
click_adid = feature_column.embedding_column(click_adid, embedding_size)

demo_seq(click_adid)


[[[ 0.640019    0.98089993  0.12807545]
  [ 0.1502139   0.9729056  -0.38220018]
  [ 0.46351585 -0.64309335 -0.06371308]
  [ 0.5411396  -0.05662565  0.28085637]
  [-0.47306636  0.8591067   0.79864115]
  [-0.63485235 -0.13707206  0.4280189 ]
  [-0.21081103 -0.3281128   0.3382318 ]
  [ 0.          0.          0.        ]]

 [[ 0.640019    0.98089993  0.12807545]
  [ 0.1502139   0.9729056  -0.38220018]
  [ 0.46351585 -0.64309335 -0.06371308]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]]]


In [13]:
# start to build linear model
varlen_max_len = 8
embedding_size = 3
backet_click_adid = 10
aid_backets = 10
cp_backets = 5

sparses = [f for f in col_type if col_type[f] in ["int32", "int64"]]
varlens = [feat for feat in col_type
                        if col_type[feat] in ('stringv', 'int32List', 'int64List', 
                                                'float32List', 'float64List')]
dense = [f for f in col_type if col_type[f] in ["float32", "float64"]]

In [14]:
sparse_feature_columns = []
varlen_feature_columns = []
dense_feature_columns = []

aid = feature_column.embedding_column(
    feature_column.categorical_column_with_hash_bucket(
      "aid", aid_backets), embedding_size)
sparse_feature_columns += [aid]

click_adid = feature_column.sequence_categorical_column_with_hash_bucket(
    'click_adid',
    backet_click_adid
)
click_adid = feature_column.embedding_column(click_adid, embedding_size)
varlen_feature_columns += [click_adid]

age = feature_column.embedding_column(
            feature_column.bucketized_column(
                feature_column.numeric_column('age')
                , boundaries = [0, 50, 60, 100]), embedding_size)
sparse_feature_columns += [age]

cp = feature_column.embedding_column(
    feature_column.categorical_column_with_identity(
      "cp", cp_backets), embedding_size)
sparse_feature_columns += [cp]

chol = feature_column.embedding_column(
    feature_column.categorical_column_with_hash_bucket(
      "chol", aid_backets, dtype=tf.dtypes.int64), embedding_size)
sparse_feature_columns += [chol]

oldpeak = feature_column.numeric_column('oldpeak')
dense_feature_columns += [oldpeak]



In [15]:
class SumPooling(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super(SumPooling, self).__init__(self, **kwargs)
  
  def build(self, input_shape):
    super(SumPooling, self).build(input_shape)

  def call(self, x, mask=None):
    if mask is None:
      raise ValueError("No mask")
    seq = tf.sequence_mask(mask, maxlen=x.shape[1])
    sum_mask = tf.tile(tf.expand_dims(seq, -1), [1, 1, x.shape[-1]])
    sum_mask = tf.cast(sum_mask, tf.dtypes.float32)
    sum_mask *= x

    res = tf.math.reduce_sum(sum_mask, axis = 1)
    return res

In [16]:
inputs = {
  "aid": layers.Input(name="aid", shape=(1,), dtype=tf.dtypes.string),
  "click_adid": layers.Input(name="click_adid", 
          shape=(varlen_max_len, ), dtype=tf.dtypes.string),
  "age": layers.Input(name="age", shape=(1,), dtype=tf.dtypes.int64),
  "cp": layers.Input(name="cp", shape=(1,), dtype=tf.dtypes.int64),
  "chol": layers.Input(name="chol", shape=(1,), dtype=tf.dtypes.int64),
  "oldpeak": layers.Input(name="oldpeak", shape=(1,), dtype=tf.dtypes.float32),
}
dense_f_ = layers.DenseFeatures(sparse_feature_columns \
                                  + dense_feature_columns)(inputs)
varlen_f_, seq_len = tf.keras.experimental \
                .SequenceFeatures(varlen_feature_columns)(inputs)
sumpooling = SumPooling()(varlen_f_, mask=seq_len)
# sumpooling = layers.Flatten()(varlen_f_)
concats = tf.keras.layers.Concatenate()([dense_f_, sumpooling])
outputs = layers.Dense(1, activation='sigmoid')(dense_f_)
model = tf.keras.Model(inputs = inputs, outputs=outputs)

# model = tf.keras.Sequential([
#   layers.DenseFeatures(chol),
#   layers.Dense(1, activation='sigmoid')
# ])

model.compile(optimizer='adam', loss=tf.losses.BinaryCrossentropy(), )
# model.build(input_shape = (None, None))
model.summary()
# tf.keras.utils.plot_model(model, "./aa.png", show_shapes=True)
# model.fit(x = example_batch, y=label, epochs = 1)


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
age (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
aid (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
chol (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
click_adid (InputLayer)         [(None, 8)]          0                                            
______________________________________________________________________________________________

In [17]:
model.fit(ds, epochs = 100, steps_per_epoch = 10)


Train for 10 steps
Epoch 1/100
10/10 [==============================] - 1s 128ms/step - loss: 0.5925
Epoch 2/100
10/10 [==============================] - 0s 8ms/step - loss: 0.6450
Epoch 3/100
10/10 [==============================] - 0s 4ms/step - loss: 0.5709
Epoch 4/100
10/10 [==============================] - 0s 5ms/step - loss: 0.6225
Epoch 5/100
10/10 [==============================] - 0s 4ms/step - loss: 0.5509
Epoch 6/100
10/10 [==============================] - 0s 5ms/step - loss: 0.6005
Epoch 7/100
10/10 [==============================] - 0s 4ms/step - loss: 0.5315
Epoch 8/100
10/10 [==============================] - 0s 4ms/step - loss: 0.5790
Epoch 9/100
10/10 [==============================] - 0s 3ms/step - loss: 0.5127
Epoch 10/100
10/10 [==============================] - 0s 3ms/step - loss: 0.5578
Epoch 11/100
10/10 [==============================] - 0s 4ms/step - loss: 0.4942
Epoch 12/100
10/10 [==============================] - 0s 4ms/step - loss: 0.5369
Epoch 13/100
10/